In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}





Preconfiguring packages ...
Selecting previously unselected package cron.
(Reading database ... 18408 files and directories currently installed.)
Preparing to unpack .../00-cron_3.0pl1-128ubuntu5_amd64.deb ...
Unpacking cron (3.0pl1-128ubuntu5) ...
Selecting previously unselected package libapparmor1:amd64.
Preparing to unpack .../01-libapparmor1_2.11.0-2ubuntu17.1_amd64.deb ...
Unpacking libapparmor1:amd64 (2.11.0-2ubuntu17.1) ...
Selecting previously unselected package libdbus-1-3:amd64.
Preparing to unpack .../02-libdbus-1-3_1.10.22-1ubuntu1_amd64.deb ...
Unpacking libdbus-1-3:amd64 (1.10.22-1ubuntu1) ...
Selecting previously unselected package dbus.
Preparing to unpack .../03-dbus_1.10.22-1ubuntu1_amd64.deb ...
Unpacking dbus (1.10.22-1ubuntu1) ...
Selecting previously unselected package dirmngr.
Preparing to unpack .../04-dirmngr_2.1.15-1ubuntu8.1_amd64.deb ...
Unpacking dirmngr (2.1.15-1ubuntu8.1) ...
Selecting previously unselected package distro-info-data.
Preparing to unpack .

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
cd drive/project_data/classifier_august/

/content/drive/project_data/classifier_august


In [0]:
ls

pascalX.npy      pascaly.npy      vgg16_weights_tf_dim_ordering_tf_kernels.h5
pascalX_val.npy  pascaly_val.npy


# Imports, Load Data

In [0]:
import keras
import tensorflow as tf
from keras.models import Model
from keras import layers
from keras.models import Sequential
from keras.layers import Reshape, Activation, Conv2D, Input, MaxPooling2D, BatchNormalization, Flatten, Dense, Lambda
from keras.layers.advanced_activations import LeakyReLU, Softmax
from keras.layers.merge import concatenate
from keras.optimizers import SGD, Adam, RMSprop
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard

import numpy as np
import os
import cv2
import math

Using TensorFlow backend.


In [0]:
X = np.load('pascalX.npy')
Y = np.load('pascaly.npy')

#X_val = np.load('pascalX_val.npy')
#Y_val = np.load('pascaly_val.npy')

print(X.shape)
print(Y.shape)

#print(X_val.shape)
#print(Y_val.shape)

(16551, 300, 300, 3)
(16551, 20)


# VGG Model 

removed extra layers, training only vgg16 with dense layer on pascal data

In [0]:
input_image = Input(shape=(300, 300, 3))

#############################################################
####### VGG16 Model  ########################################
#############################################################
# Block 1
x = layers.Conv2D(64, (3, 3),
                  activation='relu',
                  padding='same',
                  name='block1_conv1')(input_image)
x = BatchNormalization()(x)
x = layers.Conv2D(64, (3, 3),
                  activation='relu',
                  padding='same',
                  name='block1_conv2')(x)
x = BatchNormalization()(x)
x = layers.MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

# Block 2
x = layers.Conv2D(128, (3, 3),
                  activation='relu',
                  padding='same',
                  name='block2_conv1')(x)
x = BatchNormalization()(x)
x = layers.Conv2D(128, (3, 3),
                  activation='relu',
                  padding='same',
                  name='block2_conv2')(x)
x = BatchNormalization()(x)
x = layers.MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)

# Block 3
x = layers.Conv2D(256, (3, 3),
                  activation='relu',
                  padding='same',
                  name='block3_conv1')(x)
x = BatchNormalization()(x)
x = layers.Conv2D(256, (3, 3),
                  activation='relu',
                  padding='same',
                  name='block3_conv2')(x)
x = BatchNormalization()(x)
x = layers.Conv2D(256, (3, 3),
                  activation='relu',
                  padding='same',
                  name='block3_conv3')(x)
x = BatchNormalization()(x)
x = layers.MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)

# Block 4
x = layers.Conv2D(512, (3, 3),
                  activation='relu',
                  padding='same',
                  name='block4_conv1')(x)
x = BatchNormalization()(x)
x = layers.Conv2D(512, (3, 3),
                  activation='relu',
                  padding='same',
                  name='block4_conv2')(x)
x = BatchNormalization()(x)
x = layers.Conv2D(512, (3, 3),
                  activation='relu',
                  padding='same',
                  name='block4_conv3')(x)
x = BatchNormalization()(x)
x = layers.MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)

# Block 5
x = layers.Conv2D(512, (3, 3),
                  activation='relu',
                  padding='same',
                  name='block5_conv1')(x)
x = BatchNormalization()(x)
x = layers.Conv2D(512, (3, 3),
                  activation='relu',
                  padding='same',
                  name='block5_conv2')(x)
x = BatchNormalization()(x)
x = layers.Conv2D(512, (3, 3),
                  activation='relu',
                  padding='same',
                  name='block5_conv3')(x)
x = BatchNormalization()(x)

# remove the below line of pooling in detector model(when concatanating)
x = layers.MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool')(x)


"""
#############################################################
######## Skip connection  ###################################
#############################################################

skip_connection = x

x = layers.MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool')(x)
#############################################################
######## Extra Layers  ######################################
#############################################################

# Block 6
x = layers.Conv2D(512, (3, 3),
                  activation='relu',
                  padding='same',
                  name='block6_conv1')(x)
x = BatchNormalization()(x)
x = layers.Conv2D(512, (3, 3),
                  activation='relu',
                  padding='same',
                  name='block6_conv2')(x)
x = BatchNormalization()(x)
x = layers.Conv2D(512, (3, 3),
                  activation='softmax',
                  padding='same',
                  name='block6_conv3')(x)
x = BatchNormalization()(x)

# Block 7
skip_connection = Conv2D(64, (1,1), strides=(1,1), padding='same', name='conv_21', use_bias=False)(skip_connection)
skip_connection = BatchNormalization()(skip_connection)
skip_connection = Softmax(axis=-1)(skip_connection)
skip_connection = Lambda(space_to_depth_x2)(skip_connection)


# Merge
x = concatenate([skip_connection, x])

"""


# dense block
x = layers.Flatten(name='flatten')(x)
x = layers.Dense(2000, activation='relu', name='new_dense_fc1')(x)
x = layers.Dense(2000, activation='relu', name='new_dense_fc2')(x)
x = layers.Dense(20, activation='softmax', name='new_predictions')(x)




model = Model(input_image, x)

In [0]:
model.load_weights('vgg16_weights_tf_dim_ordering_tf_kernels.h5',by_name=True)

In [0]:
#optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
optimizer = SGD(lr=0.001, decay=0.0, momentum=0.9)

model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])


In [0]:
model.fit(X, Y, batch_size=24, epochs=5)

Epoch 1/5
16551/16551 [==============================] - 1217s 74ms/step - loss: 2.7792 - acc: 0.4612
Epoch 2/5
16551/16551 [==============================] - 1203s 73ms/step - loss: 1.9066 - acc: 0.6398
Epoch 3/5
16551/16551 [==============================] - 1203s 73ms/step - loss: 1.4461 - acc: 0.7264
Epoch 4/5
16551/16551 [==============================] - 1203s 73ms/step - loss: 1.3318 - acc: 0.7511
Epoch 5/5
16551/16551 [==============================] - 1203s 73ms/step - loss: 1.2698 - acc: 0.7626


In [0]:
model.save_weights('Classifier5epoch.h5')

In [0]:
model.fit(X, Y, batch_size=24, epochs=5)

Epoch 1/5
16551/16551 [==============================] - 1199s 72ms/step - loss: 1.2008 - acc: 0.7698
Epoch 2/5
16551/16551 [==============================] - 1198s 72ms/step - loss: 1.1683 - acc: 0.7719
Epoch 3/5
16551/16551 [==============================] - 1198s 72ms/step - loss: 1.1272 - acc: 0.7738
Epoch 4/5
16551/16551 [==============================] - 1199s 72ms/step - loss: 1.0907 - acc: 0.7729
Epoch 5/5
16551/16551 [==============================] - 1198s 72ms/step - loss: 1.0771 - acc: 0.7763


In [0]:
model.save_weights('Classifier10epoch.h5')

In [0]:
#optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
optimizer = SGD(lr=0.0001, decay=0.0, momentum=0.9)

model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])


In [0]:
model.fit(X, Y, batch_size=24, epochs=1)

Epoch 1/1
16551/16551 [==============================] - 1203s 73ms/step - loss: 1.0562 - acc: 0.7737


In [0]:
model.save_weights('Classifier11epoch.h5')

In [0]:
model.fit(X, Y, batch_size=24, epochs=4)

Epoch 1/4
16551/16551 [==============================] - 1200s 73ms/step - loss: 0.9949 - acc: 0.7751
Epoch 2/4
16551/16551 [==============================] - 1200s 73ms/step - loss: 0.9569 - acc: 0.7759
Epoch 3/4
16551/16551 [==============================] - 1199s 72ms/step - loss: 0.9347 - acc: 0.7759
Epoch 4/4
16551/16551 [==============================] - 1200s 72ms/step - loss: 0.9186 - acc: 0.7780


In [0]:
model.save_weights('Classifier15epoch.h5')

In [0]:
model.fit(X, Y, batch_size=24, epochs=5)
model.save_weights('Classifier20epoch.h5')

Epoch 1/5
16551/16551 [==============================] - 1199s 72ms/step - loss: 0.9090 - acc: 0.7761
Epoch 2/5
16551/16551 [==============================] - 1199s 72ms/step - loss: 0.9011 - acc: 0.7751
Epoch 3/5
16551/16551 [==============================] - 1200s 72ms/step - loss: 0.8967 - acc: 0.7766
Epoch 4/5
16551/16551 [==============================] - 1200s 73ms/step - loss: 0.8920 - acc: 0.7787
Epoch 5/5
16551/16551 [==============================] - 1199s 72ms/step - loss: 0.8890 - acc: 0.7760


In [0]:
model.fit(X, Y, batch_size=24, epochs=5)
model.save_weights('Classifier25epoch.h5')

Epoch 1/5
16551/16551 [==============================] - 1200s 73ms/step - loss: 0.8892 - acc: 0.7758
Epoch 2/5
16551/16551 [==============================] - 1199s 72ms/step - loss: 0.8857 - acc: 0.7764
Epoch 3/5
 5280/16551 [========>.....................] - ETA: 13:37 - loss: 0.8798 - acc: 0.7722

KeyboardInterrupt: ignored

In [0]:
model.fit(X, Y, batch_size=24, epochs=5)
model.save_weights('Classifier30epoch.h5')

In [0]:
model.fit(X, Y, batch_size=24, epochs=5)
model.save_weights('Classifier35epoch.h5')

In [0]:
model.fit(X, Y, batch_size=24, epochs=5)
model.save_weights('Classifier40epoch.h5')

In [0]:
model.fit(X, Y, batch_size=24, epochs=5)
model.save_weights('Classifier45epoch.h5')